In [1]:
import numpy as np
from imageanalysis_utils import Gaussian1d, GaussianRing1d
from uvanalysis_utils import plot_uv, set_grid, deproject_vis, bin_vis
from galario.double import sampleProfile

pi = np.pi
arcsec = pi / 180.0 / 3600.0  # in rad

ImportError: /home/yamato/.cache/pypoetry/virtualenvs/visibility-fit-NNZ9635L-py3.8/src/galario/build/temp.linux-x86_64-3.8/src/libgalario_single.so: undefined symbol: fftwf_alloc_real

In [ ]:
datafilepath = "/raid/work/yamato/edisk_data/analysis_data/"
uvtabfilename = datafilepath + "L1489IRS_continuum.uvtab"
visibility = np.loadtxt(uvtabfilename, unpack=True)
u, v, real, imag, weight = np.ascontiguousarray(visibility) 

In [ ]:
# model function derived from image fit
def model_func_1d(r, F_c, sigma_c, r0_r, F_r, sigma_r, F_b, sigma_b):
    return Gaussian1d(r, F_c, sigma_c) + GaussianRing1d(r, r0_r, F_r, sigma_r) + Gaussian1d(r, F_b, sigma_b)

In [ ]:
nxy, dxy, r, rmin, dr = set_grid(u, v)

model = model_func_1d(
    r / arcsec,
    9.42140643,
    0.03736141,
    0.41094978,
    8.40042786,
    0.23232904,
    9.39836001,
    1.86108534,
)

model_V = sampleProfile(
    intensity=model,
    Rmin=rmin,
    dR=dr,
    nxy=nxy,
    dxy=dxy,
    u=u,
    v=v,
    # PA=np.radians(69),
    # inc=np.radians(73),
    check=False,
)

In [ ]:
# plot observation and model
fig = plot_uv(
    u,
    v,
    real,
    imag,
    weight,
    incl=73,
    PA=69.0,
    uvrange=(0, 3000e3),
    binsize=10e3,
    fmt="o",
    capsize=3,
    markersize=5,
    zorder=-100,
)

# deprojection and binning
_, _, uvdist_deproj = deproject_vis(u, v,
    incl=73, PA=69
)

uvdist_deproj_binned, real_binned, imag_binned = bin_vis(
    uvdist=uvdist_deproj,
    real=model_V.real,
    imag=model_V.imag,
    bins=np.arange(0, 3000e3, 10e3),
    weighted_average=False,
)

for ax, V in zip(fig.axes, [real_binned, imag_binned]):
    ax.plot(uvdist_deproj_binned/1e3, V)
    ax.grid()


fig.axes[0].set(xlim=(0, 3000), ylim=(0, 0.05))
fig.axes[1].set(xlim=(0, 3000), ylim=(-0.005, 0.005))
for ax in fig.axes:
    ax.grid()
